# Fitting a matrix transformation

For each point, we have (up to measurement error),

$$
\begin{pmatrix}
  x_1&x_2&1&0&0&0\\
  0&0&0&x_1&x_2&1
\end{pmatrix}\begin{pmatrix}
  F_{11}\\
  F_{12}\\
  c_1\\
  F_{21}\\
  F_{22}\\
  c_2
\end{pmatrix}=\begin{pmatrix}
  r_1\\
  r_2
\end{pmatrix}
$$

vcat all the points gives the standard matrix form of least squares problem. The
adapter between the two problems is implemented as `fit_F` in [`lsq.jl`](lsq.jl)

1. See [$F_a$](#$F_a$) below

2. See [$F_b$](#$F_b$) and the [difference](#Difference) below.

3. See `fit_Fc` in [`lsq.jl`](lsq.jl) for the code used for this problem. $F_c$ and
    $E$ are shown [below](#$F_c$-and-$E$). The other expression for $E$ is the
    smallest two singular values of $A$ (See the assertion for $E1 \approx E2$ in
    `fit_Fc`).

    The $E$ constructed this way is guaranteed to be (one of) the smallest since the
    hcat matrix ($A'$) is garenteed to be the closest (Frobenius norm) amount rank
    $\leqslant2$ matrices by construct. And the hcat of $x''$ and $r''$ cannot have
    a rank larger than $2$.

4. See the value of differences [below](#Difference)

In [1]:
include("lsq.jl")

fit_Fc (generic function with 1 method)

In [2]:
data = readdlm("beads.txt", ' ');
x = sub(data, (:, 1:2));
r = sub(data, (:, 3:4));

#### $F_a$

In [3]:
F_a, c_a = fit_F(x, r)
F_a

2x2 Array{Float64,2}:
  1.47806   0.979544
 -0.950321  1.9868  

#### $F_b$

In [4]:
F_b, c_b = fit_F(r, x)
F_b

2x2 Array{Float64,2}:
 0.502728  -0.249489
 0.239929   0.376422

#### Difference

In [5]:
F_a - inv(F_b)

2x2 Array{Float64,2}:
 -0.0330822  -0.0220283
  0.012871   -0.0313974

#### $F_c$ and $E$

In [6]:
F_c, E1 = fit_Fc(data)

(
2x2 Array{Float64,2}:
  1.50287   0.996976
 -0.960257  2.01331 ,

0.06434391891471973)

In [7]:
# Somehow I didn't find a function for Frobenius norm
function frobenius_diff(F1, F2)
    @assert size(F1) == size(F2)
    diff2 = zero(eltype(F1))
    @inbounds @simd for i in eachindex(F1)
        diff2 += abs2(F1[i] - F2[i])
    end
    diff2
end

frobenius_diff (generic function with 1 method)

In [8]:
F_expect = [1.5 1
            -1 2]

2x2 Array{Float64,2}:
  1.5  1.0
 -1.0  2.0

#### Difference

In [9]:
@show frobenius_diff(F_expect, F_a)
@show frobenius_diff(F_expect, inv(F_b))
@show frobenius_diff(F_expect, F_c);

frobenius_diff(F_expect,F_a) = 0.0035420515657479668
frobenius_diff(F_expect,inv(F_b)) = 0.0018126172734959033
frobenius_diff(F_expect,F_c) = 0.001774043948842957
